# Code to calculate tissue composition in GT tissue map, vs pix2pix, pyramid pix2pix, and I2SB:
### Since this is to calculate bulk tissue composition so that it is invariant to the slight pixel offsets vs GT, all you have to do is just sum up the masks and then represent the tissue composition as % by dividing by the total pixel area.

In [31]:
import os
import numpy as np
from tqdm import tqdm
from PIL import Image
import cv2

In [34]:
# for i2sb, it is tmp, when finish sampling I2SB, do evaluation on all data:
gt_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\val\HE\classification_10172023"
# p2p_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\infer\pix2pix_sample1_only\classification_10162023"
gt_large_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1\val\HE\classification_11242023"

p_p2p_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1\infer\pyramid-pix2pix\classification_12262023"
i2sb_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\infer\i2sb\unconditional\test-run-4\iter_24388\classification_12262023"
i2sb_cond_pth = r"\\10.99.68.51\Kyu\IHC2HE\Balanced_Aligned\dataset_v1_256x256\infer\i2sb\conditional\test-run-seg-map-cond\iter_19656\classification_12302023"

gt_images = [os.path.join(gt_pth,x) for x in os.listdir(gt_pth) if x.endswith(".png")]
gt_large_images = [os.path.join(gt_large_pth,x) for x in os.listdir(gt_large_pth) if x.endswith(".png")]
# p2p_images = [os.path.join(p2p_pth,x) for x in os.listdir(p2p_pth) if x.endswith(".png")]
p_p2p_images = [os.path.join(p_p2p_pth,x) for x in os.listdir(p_p2p_pth) if x.endswith(".png")]
i2sb_images = [os.path.join(i2sb_pth,x) for x in os.listdir(i2sb_pth) if x.endswith(".png")]
i2sb_cond_images = [os.path.join(i2sb_cond_pth,x) for x in os.listdir(i2sb_cond_pth) if x.endswith(".png")]

In [9]:
def return_tissue_composition(image_directory):
    pixel_value_counts = np.zeros(9, dtype=int)
    for filename in tqdm(image_directory,colour="red"):
        image = cv2.imread(filename, 0)
        unique, counts = np.unique(image, return_counts=True)
        pixel_value_counts[unique] += counts
    tissue_comp = [x/pixel_value_counts.sum() for x in pixel_value_counts]
    tissue_comp = [x*100 for x in tissue_comp]
    tissue_comp = tissue_comp[1:]
    return tissue_comp

In [10]:
gt_pixel_value_counts = return_tissue_composition(gt_images)
gt_large_pixel_value_counts = return_tissue_composition(gt_large_images)
p_p2p_pixel_value_counts = return_tissue_composition(p_p2p_images)
i2sb_pixel_value_counts = return_tissue_composition(i2sb_images)

100%|██████████| 9951/9951 [01:40<00:00, 98.73it/s] 


In [36]:
i2sb_cond_pixel_value_counts = return_tissue_composition(i2sb_cond_images)

100%|██████████| 9952/9952 [01:51<00:00, 89.34it/s] 


In [37]:
def find_MAE(list1,list2):
    if len(list1) != len(list2):
        raise ValueError("Lists must be of equal length")
    absolute_differences = [abs(a - b) for a, b in zip(list1, list2)]
    mae = sum(absolute_differences) / len(absolute_differences)
    return mae

In [12]:
find_MAE(gt_large_pixel_value_counts,p_p2p_pixel_value_counts)

0.29613308584575077

In [13]:
find_MAE(gt_pixel_value_counts,i2sb_pixel_value_counts)


0.1764743117848524

In [38]:
find_MAE(gt_pixel_value_counts,i2sb_cond_pixel_value_counts)

0.07830149107807303

### Note: 1~8 in order: islet duct vessels fat acini collagen whitespace nerves

In [14]:
gt_large_pixel_value_counts[0] - p_p2p_pixel_value_counts[0]

0.2778468791311577

In [15]:
gt_pixel_value_counts[0] - i2sb_pixel_value_counts[0]

0.2679591678246587

In [39]:
gt_pixel_value_counts[0] - i2sb_cond_pixel_value_counts[0]

0.12833742466770182

In [16]:
gt_pixel_value_counts

[1.145437676040306,
 0.7418683104193096,
 1.7018703019120685,
 6.052104913153449,
 56.5369057118701,
 14.424007881875973,
 19.155930092863716,
 0.2418751118650774]

In [17]:
gt_large_pixel_value_counts

[1.288278923157327,
 0.8155905548782594,
 1.9711781161391082,
 6.332270048822237,
 56.27300992272674,
 14.037663376982955,
 18.98074962701828,
 0.30125943027508606]

In [18]:
p_p2p_pixel_value_counts

[1.0104320440261694,
 0.5238963860024209,
 1.7843640502242797,
 6.012669100255445,
 56.97376490405901,
 14.52144073903369,
 18.927915303270147,
 0.2455174731288309]

In [19]:
i2sb_pixel_value_counts

[0.8774785082156473,
 0.6823543088182877,
 1.632509029351243,
 5.783905099832931,
 56.67458121087273,
 14.869671234017387,
 19.27848848885908,
 0.20101212003269145]

In [41]:
i2sb_cond_pixel_value_counts

[1.0171002513726042,
 0.7440728006638898,
 1.8084572059165243,
 5.923803212941651,
 56.526892898167056,
 14.41092031178367,
 19.36034466292697,
 0.208408656227627]